In [1]:
!pwd

'pwd' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


# First steps using mne


## Imports

In [1]:
# std
import os
from os import path
import time
from multiprocessing import Pool
import pickle as pkl

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
import helpers
os.chdir('analysis-adelie')

%matplotlib qt

## Config

In [2]:
# This is the config. Add any value that you deem necessary. This should contain everything that belongs to the setup, 
# the filtering pipeline, etc.

cfg = {
    'paths': {
        'base': '../../data/AlphaTheta',
        'subjects': {
            'sam': {
                'prefix'
                : '/sam-AlphaTheta',
                 'recordings': {
                    'baseline': [
                        'baseline11',
                        'baseline2',
                        'baseline10'
                    ],
                    'meditation': [
                        'meditation1',
                        'meditation2'
                    ]
                },
                'channels_path': 'channelsList.txt'
            },
            'adelie': {
                'prefix': '/adelie-AlphaTheta',
                'recordings': {
                    'baseline': [
                        'baseline1',
                        'baseline2'
                    ],
                    'meditation': [
                        'meditation1',
                        'meditation2'
                    ]
                },
                'channels_path': 'channelsList.txt'
            }
        }
    },
    'columns_to_remove': [
        'TRIGGER', 'X1', 'X2', 'X3',
    ],
    'default_signal_crop': np.s_[3000:-3000], # this corresponds to ~1 second at the beginning and end, given by the sampling frequency
    'sampling_frequency': 300,
    'bands': {
        'gamma': [40, 100],
        'beta':  [12, 40],
        'alpha': [8, 12],
        'theta': [4, 8],
        'delta': [1, 4]
    }
}

NameError: name 'np' is not defined

## Helpers

In [3]:
def get_channelsList(config, subject='adelie'):
    subject_paths = helpers.get_config_value(config, 'paths', 'subjects', subject)
    base_path = helpers.get_config_value(config, 'paths', 'base')
    file_path = f"{base_path}/{subject_paths['prefix']}/offline/fif/{subject_paths['channels_path']}"
    with open(file_path, 'r') as channels_file:
        all_channels = channels_file.read().strip()
    return [channel for channel in all_channels.split('\n') if channel not in config['columns_to_remove']]

events = None
def load_raw_mne_from_fif(data_type, subject='adelie', recording=0, montage='standard_1020', config=cfg):
    global events
    """loads the data and returns an instance of mne.Raw
    
    Parameters
    ----------
    data_type : string
      type of the data, right now two options are valid: `baseline` or `meditation`
    subject: string
      name of the subject
    recording: int
      number of recording, if you have multiple of same type and subject
    montage: string
      the type of montage that was used for the recording see: https://mne.tools/dev/generated/mne.channels.make_standard_montage.html
      
    Returns
    -------
    a mne.Raw instance that has the correct montage and info and is ready to be plotted
    """
    subject_paths = helpers.get_config_value(config, 'paths', 'subjects', subject)
    base_path = helpers.get_config_value(config, 'paths', 'base')
    recording_id = helpers.get_config_value(subject_paths, 'recordings', data_type)[recording]
    file_path = f"{base_path}{subject_paths['prefix']}/offline/fif/{recording_id}-raw.fif"
    
    # Create a digitization of the montage
    digitization = mne.channels.make_standard_montage(montage)
    channels = get_channelsList(config, subject=subject)
    
    # Read from fif file
    raw = mne.io.read_raw_fif(file_path, preload=True)
    events = mne.find_events(raw)

    
    # Create info with some useful information
#     info = mne.create_info(['A2','P3', 'F8', 'C2'], sfreq=config['sampling_frequency'], ch_types='eeg')
#     raw.info = info
    
    # set the montage
    raw.set_montage(digitization)
    
    raw = raw.pick_types(eeg=True, stim=True)
    raw.set_eeg_reference('average', projection=True)#.apply_proj()
    
    return raw


NameError: name 'cfg' is not defined

## Base I/O


In [5]:
baseline_sam_pd = helpers.load_signal_data('baseline', recording=1, config=cfg)
meditation_sam_pd = helpers.load_signal_data('meditation', recording=1, config=cfg)
baseline_adelie_pd = helpers.load_signal_data('baseline', subject='adelie', recording=1, config=cfg)
meditation_adelie_pd = helpers.load_signal_data('meditation', subject='adelie', recording=1, config=cfg)

baseline_df = baseline_sam_pd
meditation_df = meditation_sam_pd

baseline_df.head()

NameError: name 'helpers' is not defined

In [13]:
from itertools import combinations 

def get_bandpower_for_electrode(signal_data, electrode, config, window_size='4s'):
    """Calculates the bandpower for the given electrode
    
    Note that this will take some time... I suggest that you only use a part of the signal to try it out.
    
    Parameters
    ----------
    signal_data: 2d pandas dataframe
        raw signal data, indexed by a timedeltaindex (or any other time-based index)
    electrode: string
        name of the electrode of interest
    config: dict
        dict of config parameters
    window_size: string
        size of rolling window
        
    Returns
    -------
    a new pandas dataframe of the bandpowers, in addition all ration combinations are listed as well
    """
    bandpowers = {}
    start = time.time()
    for band_name, band_range in config['bands'].items():
        xs = signal_data.loc[:, electrode]
        bandpowers[band_name] = helpers.bandpower(xs, config['sampling_frequency'], band_range)

    # compute all different ratios
    for bn_l, bn_r in combinations(config['bands'].keys(), 2):
        bandpowers[f"{bn_l} / {bn_r}"] = bandpowers[bn_l] / bandpowers[bn_r]
    
    end = time.time()
    print("Computed bandpower for electrode {} in {}s".format(electrode, end - start))
    return bandpowers


def get_all_electrodes_bandpowers(df, electrodes, config=cfg):
    start = time.time()
    all_bandpowers = {}
    for electrode in electrodes:
        all_bandpowers[electrode] = get_bandpower_for_electrode(df, electrode=electrode, config=config),

    end = time.time()
    print("Took {}s to compute bandpower for all electrodes".format(end-start))
    return all_bandpowers

def bp_dict_to_df(bandpower_dict):
    for electrode, bandpowers in bandpower_dict.items():
        for band, power in 
    
electrodes_to_check = get_channelsList(cfg)
bandpower_adelie = get_all_electrodes_bandpowers(baseline_adelie_pd, electrodes_to_check)
#bandpower_sam = pkl.load(open("all_bandpowers_checkpoints_sam2_delta_1Hz.pkl", "rb"))

Computed bandpower for electrode P3 in 0.024381399154663086s
Computed bandpower for electrode C3 in 0.021800518035888672s
Computed bandpower for electrode F3 in 0.024976491928100586s
Computed bandpower for electrode Fz in 0.025922775268554688s
Computed bandpower for electrode F4 in 0.024898052215576172s
Computed bandpower for electrode C4 in 0.026042699813842773s
Computed bandpower for electrode P4 in 0.026336193084716797s
Computed bandpower for electrode Cz in 0.025951147079467773s
Computed bandpower for electrode Pz in 0.025910377502441406s
Computed bandpower for electrode Fp1 in 0.02694392204284668s
Computed bandpower for electrode Fp2 in 0.023954391479492188s
Computed bandpower for electrode T3 in 0.026917695999145508s
Computed bandpower for electrode T5 in 0.025908231735229492s
Computed bandpower for electrode O1 in 0.025941848754882812s
Computed bandpower for electrode O2 in 0.02695918083190918s
Computed bandpower for electrode F7 in 0.026023149490356445s
Computed bandpower for e

In [17]:

bandpower_adelie['P3']

({'gamma': 17.37417820772515,
  'beta': 9.195151469264445,
  'alpha': 25.432897728564964,
  'theta': 16.588844095382434,
  'delta': 31.8538572506828,
  'gamma / beta': 1.8894934211578547,
  'gamma / alpha': 0.6831379732326506,
  'gamma / theta': 1.0473410991041454,
  'gamma / delta': 0.5454340449570744,
  'beta / alpha': 0.3615455685545776,
  'beta / theta': 0.5542972986179278,
  'beta / delta': 0.2886668134694219,
  'alpha / theta': 1.5331326029909644,
  'alpha / delta': 0.7984244271710548,
  'theta / delta': 0.5207797587850007},)

In [202]:
def reorder_dict(bandpower_dict):
    bp_dict = {}
    for electrode, dfs in bandpower_adelie.items():
        for recording, bandpowers in dfs.items():
            bp_dict[recording] = {}
            bp_dict[recording][electrode] = bandpowers
    return bp_dict

def average_bandpower(bandpower_dict_ordered):
    res = {}
    for recording_name, recording in bandpower_dict_ordered.items():
        res[recording_name] = {}
        for electrode_name, bandpower in recording.items():
            for band_name, values in bandpower[0].items():
                if band_name in res[recording_name]:
                    res[recording_name][band_name].add(values)
                else:
                    res[recording_name][band_name] = values
        
        for band_name, values in bandpower[0].items():
            res[recording_name][band_name] = res[recording_name][band_name] / len(bandpower[0].items())
        
    return res

def epoched_average_bandpower(average_bandpower):
    res = {}
    for recording_name, average in average_bandpower.items():
        res[recording_name] = {}
        for band_name, values in average.items():
            res[recording_name][band_name] = pd.DataFrame(values).groupby([pd.Grouper(freq='10S')]).agg(['mean', 'std'])
    return res
        
#bandpower_adelie_2 = reorder_dict(bandpower_adelie)
avg_bp = average_bandpower(bandpower_adelie)
epoched_avg_bp = epoched_average_bandpower(avg_bp)

# Bandpower by epoch

In [204]:
plt.figure()
for band, bandpower in epoched_avg_bp['baseline'].items():

    mean = bandpower.dropna()['P3']['mean']
    std = bandpower.dropna()['P3']['std']
    t = mean.reset_index().index

    plt.plot(t, mean, label=band)
    plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

plt.title("Baseline")
plt.xlabel("Epochs")
plt.ylabel("Spectral power (µV²/Hz)")  

plt.legend()
plt.show()

plt.figure()
for band, bandpower in epoched_avg_bp['meditation'].items():
    
    mean = bandpower.dropna()['P3']['mean']
    std = bandpower.dropna()['P3']['std']
    t = mean.reset_index().index

    plt.plot(t, mean, label=band)
    plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

plt.title("Meditation")
plt.xlabel("Epochs")
plt.ylabel("Mean spectral power (µV²/Hz)")    

plt.legend()
plt.show()

In [96]:
epoched_avg_bp['baseline'].keys()

dict_keys(['gamma', 'beta', 'alpha', 'theta', 'delta', 'gamma / beta', 'gamma / alpha', 'gamma / theta', 'gamma / delta', 'beta / alpha', 'beta / theta', 'beta / delta', 'alpha / theta', 'alpha / delta', 'theta / delta'])

In [159]:
plt.figure()
bandpower = epoched_avg_bp['baseline']['alpha']

mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='Alpha band')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

bandpower = epoched_avg_bp['baseline']['theta']

mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='Theta band')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

plt.title('Baseline')
plt.xlabel("Epochs")
plt.legend()
plt.show()

plt.figure()
bandpower = epoched_avg_bp['meditation']['alpha']
    
mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='Alpha band')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

bandpower = epoched_avg_bp['meditation']['theta']
    
mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='Theta band')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

plt.title('Meditation')
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [160]:
plt.figure()
bandpower = epoched_avg_bp['baseline']['alpha / theta']

mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='baseline')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

bandpower = epoched_avg_bp['meditation']['alpha / theta']
    
mean = bandpower.dropna()['T4']['mean']
std = bandpower.dropna()['T4']['std']
t = mean.reset_index().index

plt.plot(t, mean, label='meditation')
plt.fill_between(t, mean - std/2, mean + std/2, alpha=0.2)

plt.xlabel("Epochs")
plt.ylabel("Mean alpha/theta ratio")

plt.legend()
plt.show()

In [223]:
def aggregate_all_bandpowers(all_bandpowers):
    aggregated_fns = ['mean', 'std', 'min', 'max']
    aggregated_power = pd.DataFrame(index=pd.MultiIndex.from_product([
        list(all_bandpowers["baseline"].keys()),
        list(all_bandpowers["baseline"]["P3"][0].keys()),
        ['baseline', 'meditation']
    ], names=["electrode", "band/ratio", "recording_type"]), columns=aggregated_fns)
    
    for recording_name, recording in all_bandpowers.items():
        for electrode_name, bandpowers in recording.items():
            for band, power in bandpowers[0].items():
                aggregated_power.loc[(electrode_name, band, recording_name), :] = power.agg(aggregated_fns)


    for fn in aggregated_fns:
        aggregated_power[fn] = aggregated_power[fn].astype(float)
    
    return aggregated_power

In [224]:
bandpower_adelie.keys()

dict_keys(['baseline', 'meditation'])

In [225]:
aggregated_power_adelie = aggregate_all_bandpowers(bandpower_adelie)
aggregated_power_adelie.head()

mean        std       min  \
electrode band/ratio recording_type                                   
P3        gamma      baseline        17.689674  66.552068  3.133862   
                     meditation       5.417990   2.841986  2.415862   
          beta       baseline         9.275944  15.516835  2.606400   
                     meditation       6.925564   2.068583  2.386183   
          alpha      baseline        25.583483  14.794101  3.110274   

                                            max  
electrode band/ratio recording_type              
P3        gamma      baseline        646.836497  
                     meditation       30.610857  
          beta       baseline        161.908883  
                     meditation       16.790086  
          alpha      baseline         96.515982

In [226]:
mean_bp = aggregated_power_adelie.groupby(level=[1,2]).mean()
mean_bp

mean          std       min           max
band/ratio    recording_type                                                 
alpha         baseline         20.017735    12.472374  2.178596     91.198194
              meditation       29.740265   177.394021  2.051713   3785.867895
alpha / delta baseline          0.900104     0.827649  0.030163      7.436004
              meditation        1.092627     1.048860  0.012990     13.143558
alpha / theta baseline          1.249163     0.581563  0.231016      5.151058
              meditation        1.442510     0.751715  0.137467     17.919588
beta          baseline          9.130594    21.016920  2.122457    218.266467
              meditation        5.761261    10.215450  1.883213    195.108324
beta / alpha  baseline          0.630796     0.754863  0.117538     10.323249
              meditation        0.392164     0.255024  0.070524      7.132534
beta / delta  baseline          0.362623     0.407151  0.017755      5.389831
              meditation        0.310921     0.264067  0.003711      3.290174
beta / theta  baseline          0.628359     0.686717  0.093634      9.376389
              meditation        0.467991     0.272640  0.036130      4.740072
delta         baseline         85.112511   231.331728  3.560244   3159.628419
              meditation      181.620338  2856.570477  1.862771  86424.678043
gamma         baseline         26.426724   117.250766  2.817817   1134.854945
              meditation        5.709334     7.873033  2.027128    114.848422
gamma / alpha baseline          1.703118     4.754180  0.124717     68.156936
              meditation        0.474851     0.563923  0.061459     15.179534
gamma / beta  baseline          1.637542     0.968494  0.598498     16.819814
              meditation        1.036400     0.451296  0.402712      5.800822
gamma / delta baseline          0.734080     1.715158  0.025292     24.379419
              meditation        0.315392     0.354876  0.004748      5.562188
gamma / theta baseline          1.530889     4.159103  0.102295     65.947376
              meditation        0.519592     0.547108  0.032065     11.825759
theta         baseline         19.867268    17.714014  2.818621    177.568165
              meditation       45.390105   551.249139  2.207626  13617.689135
theta / delta baseline          0.668103     0.575976  0.035396      6.176625
              meditation        0.704495     0.601124  0.028491      9.338996

In [227]:
# bandpowers: mean 

plt.figure()
mean_bp[mean_bp.index.get_level_values(0).isin(['alpha','beta','theta','gamma','delta'])]['mean'].unstack(1).plot.bar(rot=0)

plt.ylabel("Mean spectral power (µV²/Hz)")

plt.legend()
plt.show()

In [178]:
# bandpowers: mean et std

bandList = ['alpha','beta','theta','gamma','delta']
mean_bp[mean_bp.index.get_level_values(0).isin(bandList)]['mean'].unstack(1).plot.bar(rot=0, yerr=mean_bp[mean_bp.index.get_level_values(0).isin(bandList)]['std'].unstack(1))

plt.ylabel("Mean spectral power (µV²/Hz)")

plt.title("Mean spectral band powers with bar errros during baseline vs. meditation")
plt.legend()
plt.show()

In [228]:
# ratios des bandpowers: mean et std

ratioList = ['gamma / beta', 'gamma / alpha', 'gamma / theta', 'gamma / delta', 'beta / alpha', 'beta / theta', 'beta / delta', 'alpha / theta', 'alpha / delta', 'theta / delta']
mean_bp[mean_bp.index.get_level_values(0).isin(ratioList)]['mean'].unstack(1).plot.bar(rot=0, yerr=mean_bp[mean_bp.index.get_level_values(0).isin(ratioList)]['std'].unstack(1))

plt.ylabel("Mean spectral power (µV²/Hz)")

plt.title("Mean spectral ratio powers with bar errros during baseline vs. meditation")
plt.legend()
plt.show()

In [124]:
baseline_df['avg'] = baseline_df.mean(axis=1)
meditation_df['avg'] = meditation_df.mean(axis=1)

meditation_epoched_df = meditation_df.groupby([pd.Grouper(freq='1S')]).agg(['mean', 'min', 'max', 'std'])
baseline_epoched_df = baseline_df.groupby([pd.Grouper(freq='1S')]).agg(['mean', 'min', 'max', 'std'])

C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [126]:
from scipy.stats import ttest_ind

ttest_ind(baseline_df['avg'], meditation_df['avg'])

Ttest_indResult(statistic=464.56922647077965, pvalue=0.0)